Idea:

Assume only few spoofing attacks

Synchronize clocks assuming all broadcast locations are correct

Estimate positions for messages based on sensor timestamps

Remove messages with high deviations between broadcast locations and estimated locations

Re-synchronize clocks and re-check broadcast locations

In [1]:
import pandas as pd
import numpy as np
import os
import re
import position_estimator
from position_estimator import GeoPoint
from collections import defaultdict
from tqdm.notebook import tqdm
import itertools

DATA_LOCATION = "E:/thesis_data/1hour_complete_22_01_10_05/"
#DATA_LOCATION = "../raw_data/1hour/"

sensors = set()
#sensors_rev = dict()
received = defaultdict(lambda: { "sensors": list(), "timeAtServer_min": 1E12, "timeAtServer_max": 0 })
sensor_received = defaultdict(list)

MSG_CUTOFF = 100000
#MSG_CUTOFF = 1

pbar = tqdm(total=MSG_CUTOFF, mininterval=1)
pbar.update(0)

discarded = defaultdict(int)
total = 0

timeAtServer_min = 1e12
timeAtServer_max = 0

for file in os.listdir(DATA_LOCATION):
    assert re.match(r"^part-\d{5}$", file)
    pbar.set_description(file)
    pbar.refresh()
    with open(os.path.join(DATA_LOCATION, file), "r") as f:
        line = f.readline()
        if not line:
            continue
        assert line == "sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence\n"
        while (line := f.readline().strip()):
            sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence = line.split(',')

            total += 1

            if sensorType in ['SBS-3', 'OpenSky', 'dump1090']:
                discarded['bad_sensortype'] += 1
                continue

            if not position_estimator.is_relevant(rawMessage):
                discarded["irrelevant"] += 1
                continue

            if 'null' in [sensorLatitude, sensorLongitude, sensorAltitude, timeAtSensor, timestamp]:
                discarded["null_values"] += 1
                continue
            
            sensorLatitude = float(sensorLatitude)
            sensorLongitude = float(sensorLongitude)
            sensorAltitude = float(sensorAltitude)
            timeAtSensor = float(timeAtSensor)
            timestamp = float(timestamp)

            timeAtServer = float(timeAtServer)
            timeAtServer_max = max(timeAtServer_max, timeAtServer)
            timeAtServer_min = min(timeAtServer_min, timeAtServer)

            # convert timestamps according to raw_data.pdf
            if sensorType == 'dump1090':
                timeAtSensor *= 89.47848533333334 # 2**30 / 12e6
                timeAtSensor += timestamp / 12e6
                assert 0 <= timestamp / 12e6 < 89.47848533333335
            elif sensorType == 'Radarcape':
                timeAtSensor += timestamp / 1e9
            else:
                discarded[f'unknown_sensortype_{sensorType}'] += 1
                continue


            #print(timestamp, timestamp/2**30)
            #assert 0 <= timestamp / 2**30 < 1

            #timeAtSensor += timestamp / 2**30

            # limit to europe for now
            if not 35 < sensorLatitude < 75:
                discarded["invalid_sensor_pos_lat"] += 1
                continue
            elif not -10 < sensorLongitude < 40:
                discarded["invalid_sensor_pos_lon"] += 1
                continue

            if not (sensorLatitude and sensorLongitude):
                discarded["invalid_sensor_pos_zero"] += 1
                continue

            if not "pos" in received[rawMessage]:
                try:
                    received[rawMessage]["pos"] = position_estimator.get_announced_pos(rawMessage, timeAtSensor)
                    assert -90 <= received[rawMessage]["pos"].lat <= 90
                    assert -180 <= received[rawMessage]["pos"].lon <= 180
                except:
                    discarded["invalid_msg_pos"] += 1
                    #print("Couldn't get position :(")
                    del received[rawMessage]
                    continue

            received[rawMessage]["timeAtServer_min"] = min(received[rawMessage]["timeAtServer_min"], timeAtServer)
            received[rawMessage]["timeAtServer_max"] = max(received[rawMessage]["timeAtServer_max"], timeAtServer)

            sensor = (GeoPoint(sensorLatitude, sensorLongitude, sensorAltitude), sensorType, sensorSerialNumber)
            if sensor not in sensors:
                #sensors_rev[sensor] = len(sensors)
                sensors.add(sensor)

            if (timeAtSensor, sensor) not in received[rawMessage]["sensors"]:
                received[rawMessage]["sensors"].append((timeAtSensor, sensor))

            sensor_received[sensor].append((timeAtSensor, timeAtServer))

    pbar.n = len(received)
    pbar.refresh()

    print("total:", total)
    print(discarded)
    print("sum discarded:", sum(discarded.values()))

    if len(received) >= MSG_CUTOFF:
        break # memory constraint


print("Sensors:", len(sensors))
print("Relevant Received Messages:", len(received))


  0%|          | 0/100000 [00:00<?, ?it/s]

total: 1367359
defaultdict(<class 'int'>, {'bad_sensortype': 1101115, 'irrelevant': 207540, 'invalid_msg_pos': 1980, 'invalid_sensor_pos_lon': 6007, 'unknown_sensortype_GRX1090': 3328, 'invalid_sensor_pos_lat': 7395, 'null_values': 7731})
sum discarded: 1335096
total: 3045417
defaultdict(<class 'int'>, {'bad_sensortype': 2451555, 'irrelevant': 463519, 'invalid_msg_pos': 2069, 'invalid_sensor_pos_lon': 13312, 'unknown_sensortype_GRX1090': 7347, 'invalid_sensor_pos_lat': 16543, 'null_values': 16797})
sum discarded: 2971142
total: 4348773
defaultdict(<class 'int'>, {'bad_sensortype': 3500651, 'irrelevant': 661724, 'invalid_msg_pos': 2095, 'invalid_sensor_pos_lon': 18986, 'unknown_sensortype_GRX1090': 10487, 'invalid_sensor_pos_lat': 23816, 'null_values': 23909})
sum discarded: 4241668
total: 5972814
defaultdict(<class 'int'>, {'bad_sensortype': 4807050, 'irrelevant': 909716, 'invalid_msg_pos': 2144, 'invalid_sensor_pos_lon': 25962, 'unknown_sensortype_GRX1090': 14401, 'invalid_sensor_pos_

In [16]:
import pickle

with open("temp_storage/timeAtServer_minmax.pickle", "wb") as f:
    pickle.dump((timeAtServer_min, timeAtServer_max), f)

with open("temp_storage/sensors.pickle", "wb") as f:
    pickle.dump(sensors, f)

with open("temp_storage/received.pickle", "wb") as f:
    pickle.dump(dict(received), f)

with open("temp_storage/sensor_received.pickle", "wb") as f:
    pickle.dump(dict(sensor_received), f)

In [3]:
import pickle
import numpy as np
import os
import re
import position_estimator
from position_estimator import GeoPoint
from collections import defaultdict
from tqdm.notebook import tqdm
import itertools

with open("temp_storage/timeAtServer_minmax.pickle", "rb") as f:
    timeAtServer_min, timeAtServer_max = pickle.load(f)

with open("temp_storage/sensors.pickle", "rb") as f:
    sensors = pickle.load(f)

with open("temp_storage/received.pickle", "rb") as f:
    received = pickle.load(f)

with open("temp_storage/sensor_received.pickle", "rb") as f:
    sensor_received = pickle.load(f)

In [2]:
import statistics

def del_sensor(sensor):
    assert type(sensor) is tuple
    assert len(sensor) == 3
    assert type(sensor[0]) is GeoPoint
    assert type(sensor[1]) is str
    sensors.remove(sensor)
    del sensor

    

print("Sensors before:", len(sensors))

faults = defaultdict(int)

for sensor, timestamps in tqdm(sensor_received.items()):
    if len(timestamps) < 2:
        del_sensor(sensor)
        faults["lt2_signals"] += 1
        continue
    server_tds = [e[1] - e[0] for e in timestamps]
    
    #server_tds_89x = [e[1] - e[0] * 89 for e in timestamps]
    #server_tds_895x = [e[1] - e[0] * 89.5 for e in timestamps]
    #server_tds_90x = [e[1] - e[0] * 90 for e in timestamps]
    #server_tds_100x = [e[1] - e[0] * 100 for e in timestamps]
    #print(statistics.variance(server_tds), statistics.variance(server_tds_89x), statistics.variance(server_tds_895x), statistics.variance(server_tds_90x))
    if statistics.variance(server_tds) > 50:
        #print(sensor)
        #print([e - server_tds[0] for e in server_tds])
        del_sensor(sensor)
        faults["variance"] += 1
        continue
            
    
    sensor_timestamps = [e[0] for e in timestamps]
    if max(sensor_timestamps) - min(sensor_timestamps) > timeAtServer_max - timeAtServer_min + 10:
        #fucky timestamps
        faults["timestamp_range"] += 1
        del_sensor(sensor)
        continue
    if len(set([e[0] for e in timestamps])) < len(timestamps):
        # non-unique timestamps -> weird sensor behavior
        #print(sensor, len(timestamps), len(set([e[0] for e in timestamps])))
        #print(timestamps)
        #print(set([e[0] for e in timestamps]))
        faults["non_unique_timestamps"] += 1
        del_sensor(sensor)
        continue
    #for i in range(len(timestamps) - 1):
    #    if timestamps[i][0] > timestamps[i+1][0] + 100:
    #        del_sensor(sensor)
    #        break
            #print(sensor)
            #print(i, timestamps[i], timestamps[i+1])
            #print("td_sensor:", timestamps[i][0] - timestamps[i+1][0], "td_server:", timestamps[i][1] - timestamps[i+1][1])

print(faults)

# clean up received data structure
for msg in tqdm(received):
    for i in reversed(range(len(received[msg]["sensors"]))):
        if received[msg]['sensors'][i][1] not in sensors:
            del received[msg]['sensors'][i]

            

print("Sensors after:", len(sensors))

Sensors before: 111


  0%|          | 0/111 [00:00<?, ?it/s]

defaultdict(<class 'int'>, {})


  0%|          | 0/107217 [00:00<?, ?it/s]

Sensors after: 111


In [4]:
for sensor in sensor_received:
    ts = defaultdict(int)
    for t in sensor_received[sensor]:
        ts[t[0]] += 1
    #print(sorted(ts.items()))

In [3]:
# remove messages that have been sent more than once
print("Messages before:", len(received))
for msg in tqdm(list(received.keys())):
    if len(set([s for t, s in received[msg]["sensors"]])) < len(received[msg]["sensors"]):
        del received[msg]
        continue
    if received[msg]["timeAtServer_max"] - received[msg]["timeAtServer_min"] > 5:
        #print("td too high!:", received[msg]["timeAtServer_max"] - received[msg]["timeAtServer_min"])
        del received[msg]
print("Messages after: ", len(received))

Messages before: 107217


  0%|          | 0/107217 [00:00<?, ?it/s]

Messages after:  106816


In [4]:
from position_estimator import GeoPoint
import traceback
import ipyparallel as ipp
import util
C = 299792458 # light speed, meters per second

n_cores = 16
n_messages_to_process = 32

time_delta = defaultdict(lambda: defaultdict(list))
to_process = [(e['pos'], e['sensors']) for e in received.values()]#[:n_messages_to_process]
#print(to_process[0])
with ipp.Cluster(n=n_cores) as rc:
    view = rc.load_balanced_view()
    asyncresult = view.map_async(util.calc_timedeltas, *zip(*to_process))
    #asyncresult.wait_interactive()
    for td in tqdm(asyncresult.get()):
        for s1 in td:
            for s2 in td[s1]:
                time_delta[s1][s2].extend(td[s1][s2])



'''for msg in tqdm(received):
    if "pos" not in received[msg] or received[msg]["pos"] is None:
        continue

    if len(received[msg]["sensors"]) < 2:
        continue

    #for t, s in list(received[msg]["sensors"]):
    #    dists = sorted([s[0].dist(e[1][0]) for e in received[msg]["sensors"] if e[1] != s])
    #    if dists[int(len(dists)/10)] > 1e6: # ge 1000 km
    #        print("removing:", t, s, dists)
    #        received[msg]["sensors"].remove((t, s))

    try:
        sensor_dists_to_msg_origin = {x[1]: received[msg]["pos"].dist(x[1][0]) for x in received[msg]["sensors"]}
        time_to_sensor = { s: x / C for s, x in sensor_dists_to_msg_origin.items() }
    except:
        traceback.print_exc()
        print(received[msg]["pos"].lat, received[msg]["pos"].lon, received[msg]["pos"].alt)
        print({x[1]: (x[1][0].lat, x[1][0].lon, x[1][0].alt) for x in received[msg]["sensors"]})

    for (t1, s1), (t2, s2) in itertools.combinations(received[msg]["sensors"], 2):
        assert s1 != s2
        td = (t1 - time_to_sensor[s1]) - (t2 - time_to_sensor[s2])
        time_delta[s1][s2].append(td)
        time_delta[s2][s1].append(-td)
'''


lens = []
for s1 in time_delta:
    for s2 in time_delta[s1]:
        lens.append(len(time_delta[s1][s2]))

np.histogram(lens)

C:\Users\Lukas Baege\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipyparallel\util.py:609: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if V(jupyter_client.__version__) < V('5.0'):


Starting 16 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/16 [00:00<?, ?engine/s]

In [20]:
import pickle


with open("temp_storage/received_filtered.pickle", "wb") as f:
    pickle.dump(dict(received), f)

with open("temp_storage/sensors_filtered.pickle", "wb") as f:
    pickle.dump(sensors, f)

with open("temp_storage/time_delta.pickle", "wb") as f:
    pickle.dump(dict(time_delta), f)

In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import re
import position_estimator
from position_estimator import GeoPoint
from collections import defaultdict
from tqdm.notebook import tqdm
import itertools
from position_estimator import GeoPoint
import traceback
import ipyparallel as ipp
import util

with open("temp_storage/received_filtered.pickle", "rb") as f:
    received = pickle.load(f)

with open("temp_storage/sensors_filtered.pickle", "rb") as f:
    sensors = pickle.load(f)

with open("temp_storage/time_delta.pickle", "rb") as f:
    time_delta = pickle.load(f)
    #print(time_delta)

C:\Users\Lukas Baege\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipyparallel\util.py:609: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if V(jupyter_client.__version__) < V('5.0'):


In [21]:
import statistics
time_delta_gaussians =  defaultdict(lambda: defaultdict(lambda: None))

def calc_gaussian(i, j):
    if len(time_delta[i][j]) > 1:
        mean = statistics.mean(time_delta[i][j])
        var = statistics.variance(time_delta[i][j], xbar=mean)
        if var <= 1e-9 or True:
            time_delta_gaussians[i][j] = (mean, var)
            time_delta_gaussians[j][i] = (-mean, var)


# initialize with directly known time deltas
ij_collection = itertools.combinations(time_delta.keys(), 2)
print("Calculating Gaussians")
for i, j in tqdm(ij_collection):
    calc_gaussian(i, j)

Calculating Gaussians


0it [00:00, ?it/s]

In [22]:
variances = [ time_delta_gaussians[i][j][1] for i,j in itertools.combinations(time_delta.keys(), 2) if i in time_delta_gaussians and j in time_delta_gaussians[i] and time_delta_gaussians[i][j] is not None ]
max_variance = max(variances)
print(max_variance)
done = False
for s1 in time_delta_gaussians:
    for s2 in time_delta_gaussians[s1]:
        if time_delta_gaussians[s1][s2][1] == max_variance:
            print(s1)
            print(s2)
            for msg in received:
                tmp = [e[1] for e in received[msg]["sensors"]]
                if s1 in tmp and s2 in tmp:
                    print(msg)
                    print(received[msg]["pos"])
                    print(received[msg]["timeAtServer_min"], received[msg]["timeAtServer_max"], received[msg]["timeAtServer_max"] - received[msg]["timeAtServer_min"])
                    print([e[0] for e in received[msg]["sensors"] if e[1] in [s1, s2]])
            done = True
            break
    if done:
        break


56.822736225255184
(Latitude: 49.15917, Longitude: 15.1995, Altitude: 546.5, 'dump1090', '-1408236843')
(Latitude: 48.180073, Longitude: 16.328741, Altitude: 204.0, 'dump1090', '-1408233302')
8d77058c58cd8777576e175a7710
Latitude: 47.99881, Longitude: 15.83253, Altitude: 12192.0
1641790895.001 1641790895.56 0.5590000152587891
[3487087.8158542505, 6267019.5422651665]
8d77058c58cd800d5b738a631649
Latitude: 48.07823, Longitude: 15.53096, Altitude: 12192.0
1641790996.728 1641790997.252 0.5240001678466797
[3487189.5489970837, 6267110.614837]


We now have all time deltas between stations that picked up the same signal.
We model the clock shifts using gaussian distributions, with some mean mu and standard deviation sigma

Then, to get time delta probability distribution between two stations that didn't measure the same message, we convolute the probability density functions of stations with directly known time-delta

Example:
sensors A and B measured the same 100 messages, so we can directly model their time delta distribution function as: td_AB = Gauss(mu_AB, sigma_AB)
Similarly, assume we estimate the density function for sensors B and C as Gauss(mu_BC, sigma_BC)

Now, to get the density function for sensors A and C, we can convolute these two density functions:
density function of time delta A-C = Gauss(mu_AB, sigma_AB) * Gauss(mu_BC, sigma_BC) = Gauss(mu_AB + mu_BC, sqrt(sigma_AB^2 + sigma_BC^2))

We do this for all paths going from A to C, and sum up all PDFs to get the final PDF for the time delta between A and C.
For efficiency, we could just do a "shortest path", where the path lengths are the sum of variances.
This way, we just take the "best" path, with least uncertainty.

In [23]:
n_cores = 16
import concurrent.futures

def floyd_adjust_min(k, i, j):
    #if i == j or j == k or k == i:
    #    return i, j, None
    #if i not in time_delta_gaussians:
    #    print(1, k, i, j)
    #    return
    #if k not in time_delta_gaussians:
    #    print(2, k, i, j)
    #    return
    #if k not in time_delta_gaussians[i]:
    #    print(3, k, i, j)
    #    return
    #if j not in time_delta_gaussians[k]:
    #    print(4, k, i, j)
    #    return
    if i not in time_delta_gaussians or k not in time_delta_gaussians or k not in time_delta_gaussians[i] or j not in time_delta_gaussians[k]:
        return i, j, None
    if time_delta_gaussians[i][k] is not None and time_delta_gaussians[k][j] is not None:
        new_variance = time_delta_gaussians[i][k][1] + time_delta_gaussians[k][j][1]
        if time_delta_gaussians[i][j] is None or time_delta_gaussians[i][j][1] > new_variance:
            new_mean = time_delta_gaussians[i][k][0] + time_delta_gaussians[k][j][0]
            time_delta_gaussians[i][j] = (new_mean, new_variance)
            time_delta_gaussians[j][i] = (-new_mean, new_variance)
            #print(i, j, new_mean, new_variance)


# floyd's algorithm
print("Doing Floyd's algorithm")
for k in tqdm(time_delta.keys()):
    #tdg = dict(time_delta_gaussians)
    ij_collection = [(i, j) for i, j in itertools.combinations(time_delta.keys(), 2)]
    #print(len(kij_collection))
    #asyncresult = view.map_async(floyd_adjust_min, kij_collection)
    #asyncresult.wait_interactive(return_when=concurrent.futures.ALL_COMPLETED)
    #for i, j, val in asyncresult.get():
    #    if val is None:
    #        continue
    #    time_delta_gaussians[i][j] = val
    for i, j in ij_collection:
        floyd_adjust_min(k, i, j)

#print(time_delta_gaussians)
tdg_dict = dict()
for s1 in time_delta_gaussians:
    if s1 not in tdg_dict:
        tdg_dict[s1] = dict()
    for s2 in time_delta_gaussians[s1]:
        if time_delta_gaussians[s1][s2] is None:
            continue
        tdg_dict[s1][s2] = time_delta_gaussians[s1][s2]

with open("temp_storage/time_delta_gaussians.pickle", "wb") as f:
    pickle.dump(tdg_dict, f)


Doing Floyd's algorithm


  0%|          | 0/766 [00:00<?, ?it/s]

In [10]:
time_delta_gaussians = pickle.load(open("temp_storage/time_delta_gaussians.pickle", "rb"))

In [24]:
variances = [ time_delta_gaussians[i][j][1] for i,j in itertools.combinations(time_delta.keys(), 2) if i in time_delta_gaussians and j in time_delta_gaussians[i] and time_delta_gaussians[i][j] is not None ]
num_conns = [ len(time_delta[i][j]) for i,j in itertools.combinations(time_delta.keys(), 2) if len(time_delta[i][j]) > 0]

print("Mean Variance:", statistics.mean(variances))
print("Median variance:", statistics.median(variances))
#print("Variance modes:", statistics.multimode(variances))

print("Mean nConnections:", statistics.mean(num_conns))
print("Median nConnections:", statistics.median(num_conns))
print("nConnections modes:", statistics.multimode(num_conns))

Mean Variance: 0.004229081859117333
Median variance: 2.064691744487094e-10
Mean nConnections: 266.2089818489943
Median nConnections: 93.0
nConnections modes: [1]


In [25]:
max_variance = max(variances)
print(max_variance)
done = False
for s1 in time_delta_gaussians:
    for s2 in time_delta_gaussians[s1]:
        if time_delta_gaussians[s1][s2][1] == max_variance:
            print(s1)
            print(s2)
            for msg in received:
                tmp = [e[1] for e in received[msg]["sensors"]]
                if s1 in tmp and s2 in tmp:
                    print(msg)
                    print(received[msg]["pos"])
                    print(received[msg]["timeAtServer_min"], received[msg]["timeAtServer_max"], received[msg]["timeAtServer_max"] - received[msg]["timeAtServer_min"])
                    print([e[0] for e in received[msg]["sensors"] if e[1] in [s1, s2]])
            done = True
            break
    if done:
        break


1.5607951553820025
(Latitude: 49.031386, Longitude: 2.06335, Altitude: 28.0, 'dump1090', '1408234760')
(Latitude: 46.973057, Longitude: 18.916591, Altitude: 147.0, 'dump1090', '-1408235633')


Using those time deltas, we can now check the aircraft locations

In [29]:
from dataclasses import dataclass
import pyproj
import warnings
import math
warnings.filterwarnings("ignore")

M = np.diag([1, 1, 1, -1])

@dataclass
class Vertexer:

    nodes: np.ndarray

    # Defaults
    v = 299792458

    def __post_init__(self):
        # Calculate valid input range
        max = 0
        min = 1E+10
        centroid = np.average(self.nodes, axis = 0)
        for n in self.nodes:
            dist = np.linalg.norm(n - centroid)
            if dist < min:
                min = dist

            for p in self.nodes:
                dist = np.linalg.norm(n - p)

                if dist > max:
                    max = dist

        max /= self.v
        min /= self.v

        #print(min, max)

    def errFunc(self, point, times):
        # Return RSS error
        error = 0

        for n, t in zip(self.nodes, times):
            error += ((np.linalg.norm(n - point) / self.v) - t)**2

        return error

    def find(self, times):
        def lorentzInner(v, w):
            # Return Lorentzian Inner-Product
            return np.sum(v * (w @ M), axis = -1)

        A = np.append(self.nodes, times * self.v, axis = 1)
        #print(A)
        At = np.transpose(A)
        #print("At")
        #print(At)
        AtA = np.matmul(At, A)
        #print("AtA")
        #print(AtA)
        invAtA = np.linalg.inv(AtA)
        #print("invAtA")
        #print(invAtA)
        A_plus = np.matmul(invAtA, At)
        #print("A_plus")
        #print(A_plus)


        b = 0.5 * lorentzInner(A, A)
        #print("b")
        #print(b)
        #oneA = np.linalg.solve(A_plus, np.ones(4))
        #invA = np.linalg.solve(A_plus, b)

        oneA_plus = np.matmul(A_plus, np.ones(len(self.nodes)))
        invA_plus = np.matmul(A_plus, b)

        #print("oneA_plus", oneA_plus.shape, np.ones(len(self.nodes)).shape)
        #print(oneA_plus)
        #print("invA_plus")
        #print(invA_plus)


        solution = []
        for Lambda in np.roots([ lorentzInner(oneA_plus, oneA_plus),
                                (lorentzInner(oneA_plus, invA_plus) - 1) * 2,
                                 lorentzInner(invA_plus, invA_plus),
                                ]):
            #X, Y, Z, T = M @ np.linalg.solve(, Lambda * np.ones(len(self.nodes)) + b)
            X, Y, Z, T = np.matmul(A_plus, (b + Lambda * np.ones(len(self.nodes))))
            if any(np.iscomplex([X, Y, Z, T])):
                continue
            solution.append(np.array([X,Y,Z]))
            #print("Candidate:", X, Y, Z, math.sqrt(X**2 + Y**2 + Z**2))

        if not len(solution):
            return
        #print()
        #print()
        return min(solution, key = lambda err: self.errFunc(err, times))

def to_ecef(geopoint):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    x, y, z = pyproj.transform(lla, ecef, geopoint.lon, geopoint.lat, geopoint.alt, radians=False)
    #print("to_ecef:", geopoint, "->", x, y, z)
    return (x, y, z)

def to_wgs84(x, y, z):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    lon, lat, alt = pyproj.transform(ecef, lla, x, y, z, radians=False)
    return (lat, lon, alt)


iterations = 0
for msg, data in tqdm(received.items()):
    if len(data["sensors"]) < 4:
        continue

    iterations += 1
    if iterations >= 1000:
        break

    td_base = None


    best_variances = list()
    for t1, s1 in data["sensors"]:
        if s1 not in time_delta_gaussians:
            continue
        var_sum = 0
        non_none = 0
        for t2, s2 in data["sensors"]:
            if s1 == s2:
                continue
            if s2 in time_delta_gaussians[s1]:
                non_none += 1
                var_sum += time_delta_gaussians[s1][s2][1]
        best_variances.append((-non_none, var_sum, t1, s1))

    best_variances.sort()
    #print(best_variances)

    if best_variances[0][0] > -4:
        # less than four connected sensors
        continue

    if best_variances[3][1] > 1e-6:
        # not accurate enough
        continue

    sensor_positions = list()
    timestamps = list()

    for _, _, timestamp, sensor in best_variances:
        ecef_coords = to_ecef(sensor[0])
        if td_base is None:
            td_base = sensor
            td = 0
        else:
            if sensor in time_delta_gaussians[td_base] and time_delta_gaussians[td_base][sensor] is not None:
                td = time_delta_gaussians[td_base][sensor][0]
            
        sensor_positions.append(ecef_coords)
        timestamps.append(timestamp + td)
        if len(timestamps) == 6:
            break
    
    timestamps = [ e - timestamps[0] for e in timestamps ]
    #sensor_basepos = sensor_positions[0]
    #sensor_positions = [ e - sensor_basepos for e in sensor_positions ]

    #print(sensor_positions)
    #print(timestamps)
    
    myVertexer = Vertexer(np.array(sensor_positions))
    try:
        target_ecef = myVertexer.find(np.array([[e] for e in timestamps]))
        #target_ecef += sensor_basepos
        print(target_ecef)
        target_pos = GeoPoint(*to_wgs84(*target_ecef))
    except:
        target_ecef = myVertexer.find(np.array([[e] for e in timestamps]))
        target_pos = GeoPoint(*to_wgs84(*target_ecef))
        print("Fail")
        continue

    #print(target_ecef)
    print("Sensors:")
    for t, s in zip(timestamps, sensor_positions):
        print(t,s)
    print("Variances:", [e[1] for e in best_variances[:4]])

    print("Estimated Signal location:", target_ecef)
    print("True Signal location:     ", to_ecef(data["pos"]))
    print("Dist:", np.linalg.norm(np.array(target_ecef) - np.array(to_ecef(data["pos"]))))




  0%|          | 0/105908 [00:00<?, ?it/s]

[3929347.38323298+17797.24358111j  288158.62916994 +1357.67547611j
 4999168.02562448+22762.82980613j]


TypeError: input must be a scalar

In [9]:
import pyproj
import warnings
import math
warnings.filterwarnings("ignore")



def to_ecef(geopoint):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    x, y, z = pyproj.transform(lla, ecef, geopoint.lon, geopoint.lat, geopoint.alt, radians=False)
    return (x, y, z)

def to_wgs84(x, y, z):
    ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
    lla = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
    lon, lat, alt = pyproj.transform(ecef, lla, x, y, z, radians=False)
    return (lat, lon, alt)

def lorentz_inner(u, v):
    assert len(u) == len(v) == 4
    return u[0]*v[0] + u[1]*v[1] + u[2]*v[2] - u[3]*v[3]


iterations = 0

for msg, data in tqdm(received.items()):
    #print(time_delta_gaussians)
    #break

    if len(data["sensors"]) < 4:
        continue

    iterations += 1
    if iterations >= 5:
        break

    #valid_sensors = [ data["sensors"][0] ]
    #for s in data["sensors"][1:]:
    #    if s[]

    # Bancroft method
    B = list()
    a = list()
    td_base = None


    best_variances = list()
    for t1, s1 in data["sensors"]:
        var_sum = 0
        non_none = 0
        for t2, s2 in data["sensors"]:
            if s1 == s2:
                continue
            if s2 in time_delta_gaussians[s1]:
                non_none += 1
                var_sum += time_delta_gaussians[s1][s2][1]
        best_variances.append((-non_none, var_sum, t1, s1))

    best_variances.sort()

    if best_variances[0][0] > -4:
        # less than four connected sensors
        continue

    for _, _, timestamp, sensor in best_variances:
        ecef_coords = to_ecef(sensor[0])
        if td_base is None:
            td_base = sensor
            td = 0
        else:
            if sensor in time_delta_gaussians[td_base] and time_delta_gaussians[td_base][sensor] is not None:
                td = time_delta_gaussians[td_base][sensor][0]
            
        print(timestamp, td, td*C)
        B.append([*ecef_coords, C*(-timestamp - td)])
        a.append(0.5*(ecef_coords[0]**2 + ecef_coords[1]**2 + ecef_coords[2]**2 - td**2))
        if len(a) >= 4:
            break


    e = [1] * 4#len(data["sensors"])

    print(B)
    B_plus = np.matmul(np.linalg.inv(np.matmul(np.transpose(B), B)), np.transpose(B))
    lambda_coefs = (lorentz_inner(np.matmul(B_plus,e), np.matmul(B_plus,e)), 2*(lorentz_inner(np.matmul(B_plus, a), np.matmul(B_plus,e)) - 1), lorentz_inner(np.matmul(B_plus,a), np.matmul(B_plus,a)))
    lambda_roots = np.roots(lambda_coefs)

    print(lambda_coefs)
    print(lambda_roots)
    #if len(lambda_roots) != 2:
    #    continue
    assert len(lambda_roots) == 2

    best_dist = 1e9
    for root in lambda_roots:
        sol = np.matmul(B_plus, (a + root * np.array(e)))
        print(sol)
        dist = GeoPoint(*to_wgs84(*sol[:3])).dist(data["pos"])
        best_dist = min(best_dist, dist)

    print("Distance:", best_dist)
    

    


  0%|          | 0/101230 [00:00<?, ?it/s]

107934.77944910247 0 0
27002.990904231556 80931.7889373966 24262739935879.336
39.83166457153857 107894.94786448932 32346091626077.105
5500.332380584441 102434.44722814579 30709074718397.113
[[3836217.527465573, 665602.7446539975, 5035003.349590785, -32358032834734.316], [3733262.3243191987, 664027.3054950554, 5111427.4524335265, -32358032952410.555], [3817051.941483905, 677198.9789480256, 5047900.4088057615, -32358032858705.24], [3857557.2049890393, 712177.6983716807, 5012473.618069542, -32358032882589.51]]
(2.7176898806900472e-24, -2.0000391783716394, 145337506953428.06)
[7.35933556e+23 7.26673300e+13]
[ 6.03090190e+11  3.25442863e+11  1.00137540e+12 -2.27432197e+10]
[-7.07429151e+06 -1.51480945e+06 -9.64336103e+06 -5.24209481e+00]
Distance: 20657006.93331343
11318.211749547161 0 0
1318.0034147165716 10000.208766541553 2997987166634.64
23032.327934230678 -11714.115262612577 -3511803407873.94
39144.55562034063 -27826.344505311892 -8342128216402.246
[[4066727.8986526043, 588886.56846944

In [11]:
import scipy.optimize

def estimate_position(sensors, time_delta_gaussians):
    assert len(sensors) >= 4

    # we want to find a position p, for which we want to minimize some objective function.
    # this objective function is subject to the time deltas,
    # and the distances from the estimated position and the sensor positions
    # also, we should consider the variances in the time delta distributions:
    # if some time delta probability distribution has very big variance,
    # we can't trust that sensor to the same degree as a sensor with a small time delta variance

    # attempt 1:
    def residual_error(p):
        p = position_estimator.GeoPoint(*p)
        #print("p:", p.lat, p.lon, p.alt)
        err = 0
        for i,j in itertools.combinations(range(len(sensors)), 2):
            if time_delta_gaussians[sensors[i]][sensors[j]] is None:
                continue

            dist_i = p.dist(sensors[sensors[i]][0])
            dist_j = p.dist(sensors[sensors[j]][0])
            t_i = sensors[i][0] - dist_i / C
            t_j = sensors[j][0] - dist_j / C
            #print(sensors[i], sensors[j], t_i, t_j, time_delta_gaussians[sensors[i]][sensors[j]])
            e = (t_i - t_j - time_delta_gaussians[sensors[i]][sensors[j]][0]) ** 2 / time_delta_gaussians[sensors[i]][sensors[j]][1]
            err += e
        #print(err)
        return err

    def residual_error2(p):
        p = position_estimator.GeoPoint(*p)
        err = 0
        true_t = sensor_timestamps[0] - p.dist(sensors[0][1][0]) / C
        for i in range(1, len(sensors)):
            t = sensors[i][0] - p.dist(sensors[sensors[i]][0]) / C
            err += true_t - t - time_delta_gaussians[sensors[0]][sensors[i]][0]
        #print(err, len(sensors), err/len(sensors))
        return err

    def residual_error3(p):
        p = position_estimator.GeoPoint(*p)
        err = 0
        for assumed_true in range(len(sensors)):
            true_t = sensor_timestamps[sensors[assumed_true]] - p.dist(sensors[sensors[assumed_true]][0]) / C
            for i in range(len(sensors)):
                if i == assumed_true:
                    continue
                t = sensors[i][0] - p.dist(sensors[sensors[i]][0]) / C
                err += true_t - t - time_delta_gaussians[sensors[assumed_true]][sensors[i]][0]
            #print(err, len(sensors), err/len(sensors))
            return err

    def residual_error4(p):
        p = position_estimator.GeoPoint(*p)
        # choose 4 best time_delta distributions (smallest variances)
        val = (100000, None)
        for a, b, c, d in itertools.combinations(range(len(sensors)), 4):
            s = sum([time_delta_gaussians[sensors[e]][sensors[f]][1] for e,f in itertools.combinations([a,b,c,d],2)])
            val = min(val, (s, (a,b,c,d)))
        err = 0
        for i,j in itertools.combinations(val[1], 2):
            #print(i,j, sensors[i], sensors[j])
            ti = sensors[i][0] - p.dist(sensors[sensors[i]][0]) / C
            tj = sensors[j][0] - p.dist(sensors[sensors[j]][0]) / C
            err += ti - tj - time_delta_gaussians[sensors[i]][sensors[j]][0]
        return err


    bounds = scipy.optimize.Bounds([-90, -180, 0], [90, 180, 100000])
    method = 'L-BFGS-B'
    optimum = scipy.optimize.minimize(residual_error, [sensors[0][1][0].lat, sensors[0][1][0].lon, sensors[0][1][0].alt], method=method, bounds=bounds).x
    print("optimum:", optimum)
    return GeoPoint(*optimum)


it = 0
for msg in tqdm(received):
    if len(received[msg]["sensors"]) < 4:
        continue
    
    estimated_pos = estimate_position(received[msg]["sensors"], time_delta_gaussians)
    received[msg]["estimated_pos"] = estimated_pos
    print("Broadcast pos:", received[msg]["pos"].lat, received[msg]["pos"].lon, received[msg]["pos"].alt)
    print("Estimated pos:", received[msg]["estimated_pos"].lat, received[msg]["estimated_pos"].lon, received[msg]["estimated_pos"].alt)
    print("Dist:", received[msg]["pos"].dist(received[msg]["estimated_pos"]))

    it += 1
    if it >= 10:
        break
    

  0%|          | 0/14297 [00:00<?, ?it/s]

optimum: [ 52.14    10.4601 120.    ]
Broadcast pos: 52.67409 9.99306 10972.800000000001
Estimated pos: 52.14 10.4601 120.0
Dist: 68263.95609740308
optimum: [  46.8914239    7.499006  1713.       ]
Broadcast pos: 49.28895 7.45827 10363.2
Estimated pos: 46.8914239 7.4990060000000085 1713.0
Dist: 266743.4218124952
optimum: [51.6303442  4.9487092 45.       ]
Broadcast pos: 51.87618 7.56424 12192.0
Estimated pos: 51.6303442 4.9487092 45.0
Dist: 183067.81057760966
optimum: [ 50.03496   8.24124 312.     ]
Broadcast pos: 49.32985 7.43845 10972.800000000001
Estimated pos: 50.03496 8.24124 312.0
Dist: 98082.82992399957
optimum: [ 51.43095016   6.92255878 119.56099701]
Broadcast pos: 51.43611 7.27287 3268.98
Estimated pos: 51.43095016479492 6.922558784484863 119.56099700927734
Dist: 24570.580713169555
optimum: [51.9899  4.3754 30.    ]
Broadcast pos: 51.9912 -1.62483 11887.2
Estimated pos: 51.9899 4.3754 30.0
Dist: 412224.2846733337
optimum: [51.282448  6.555248 85.      ]
Broadcast pos: 49.1738